# Vector Context Integration Test
This notebook demonstrates retrieving vector context (cue cards, adaptive prompts, and similar conversations) using the existing `EnhancedConversationDB` class.

In [ ]:
# Initialize the database
from program_files.database.enhanced_conversation_db import EnhancedConversationDB
db = EnhancedConversationDB()
print("Database initialized. Total records:", db.conversations.count())

In [ ]:
# Helper: pretty-print JSON
import json, pprint
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
# Query the vector database for context related to a medical topic
query = 'diabetes management'
context = db.get_vector_context(query, top_k=3)
if context:
    print('Cue cards:', len(context.get('relevant_cue_cards', [])))
    print('Adaptive prompts:', len(context.get('relevant_prompts', [])))
    print('Similar conversations:', len(context.get('similar_conversations', [])))
    pp.pprint(context)
else:
    print('No context found for query.')

---
Change the `query` string above to test other medical topics (e.g., `hypertension`, `blood pressure`, `medication`).